## Creates the doc2vec vector embeddings for a specific configuration

In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import time
from collections import namedtuple
import cPickle as pickle
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random

from multiprocessing import Process, Queue

from multiprocessing.dummy import Pool as ThreadPool
import itertools

from sklearn.metrics import coverage_error
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.preprocessing import MultiLabelBinarizer

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info
from functools import partial

from thesis.utils.metrics import *

## Global variables used throughout the script

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234

In [4]:
MIN_WORD_COUNT = 200
NUM_CORES = 8

In [5]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 
                                         'SVM_MODEL_NAME', 'NN_MODEL_NAME'])

In [6]:
VOCAB_MODEL = "vocab_model"
MODEL_PREFIX = "model"
VALIDATION_DICT = "validation_dict.pkl"
TEST_MATRIX = "test_matrix.pkl"
TEST_DICT = "test_dict.pkl"
METRICS = "metrics.pkl"
CLASSIFIER = "classifier.pkl"
TYPE_CLASSIFIER= "{}_classifier.pkl"

In [25]:
root_location = "/mnt/virtual-machines/data/"
exports_location = root_location + "exported_data/"

doc2vec_model_save_location = os.path.join(root_location, "parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks_levels_0_1", "full")
if not os.path.exists(doc2vec_model_save_location):
    os.makedirs(doc2vec_model_save_location)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL)):
    os.makedirs(os.path.join(doc2vec_model_save_location, VOCAB_MODEL))

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

preprocessed_location = root_location + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks/"

training_preprocessed_files_prefix = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
validation_preprocessed_files_prefix = root_location + "preprocessed_data_validation/extended_pv_abs_desc_claims_full_chunks/" + "extended_pv_validation_docs_data_preprocessed-"
test_preprocessed_files_prefix = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

## Load general data required for classification

In [8]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 19.9 s, sys: 1.22 s, total: 21.1 s
Wall time: 21.1 s


In [9]:
len(training_docs_list)

1286325

In [10]:
len(validation_docs_list)

321473

In [11]:
len(test_docs_list)

401877

# Utility functions for data loading

In [12]:
def ensure_disk_location_exists(location):
    if not os.path.exists(location):
        os.makedirs(location)

In [13]:
VALIDATION_MINI_BATCH_SIZE = 10000
def get_extended_docs_only_with_inference(doc2vec_model, file_to_write, preprocessed_files_prefix):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation or test documents
    """

    def infer_one_doc(doc_tuple):
        # doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)

    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)):
        info("===== Loading inference vectors")
        inference_documents_reps = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)))
        info("Loaded inference vectors matrix")
    else:
        inference_documents_reps = {}
        info("===== Getting vectors with inference")

        # Multi-threaded inference
#         inference_docs_iterator = ExtendedPVDocumentBatchGenerator(preprocessed_files_prefix, batch_size=None)
        inference_docs_iterator = BatchClass(preprocessed_files_prefix, batch_size=None)
        generator_func = inference_docs_iterator.__iter__()
        pool = ThreadPool(NUM_CORES)
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        mini_batch_size = VALIDATION_MINI_BATCH_SIZE
        while True:
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {} tags".format(str(inference_docs_iterator.curr_index + 1)))
            if threaded_reps_partial:
                # threaded_reps.extend(threaded_reps_partial)
                inference_documents_reps.update(threaded_reps_partial)
            else:
                break

        pickle.dump(inference_documents_reps,
                    open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write), 'w'))

    return inference_documents_reps

In [15]:
class ExtendedPVDocumentBatchGeneratorMine(object):
    def __init__(self, filename_prefix, batch_size=10000 ):
        """
        batch_size cant be > 10,000 due to a limitation in doc2vec training, 
        None means no batching (only use for inference)
        """
        assert batch_size <= 10000 or batch_size is None
        self.filename_prefix = filename_prefix
        self.curr_lines = []
        self.curr_docids = []
        self.batch_size = batch_size
        self.curr_index = 0
        self.curr_doc_index = 0
        self.batch_end = -1
    def load_new_batch_in_memory(self):
        del self.curr_lines, self.curr_docids
        self.curr_lines, self.curr_docids = [], []
        info("Loading new batch for index: {}".format(self.curr_doc_index))
        #if self.curr_doc_index > 0: self.curr_doc_index -= 1
        true_docs_count = 0
        try:
            with open(self.filename_prefix + str(self.curr_doc_index)) as preproc_file:
                for line in preproc_file:
                    line_array = line.split(" ")
                    doc_id = line_array[0]
                    doc_tokens = line_array[1:]
                    self.curr_lines.append(doc_tokens)
                    self.curr_docids.append(doc_id)
                    if is_real_doc(doc_id):
                        true_docs_count+= 1
            self.batch_end = self.curr_doc_index + true_docs_count - 1 
            info("Finished loading new batch of {} documents".format(true_docs_count))
        except IOError:
            info("No more batches to load, exiting at index: {}".format(self.curr_index))
            raise StopIteration()
    def __iter__(self):
        while True:
            if self.curr_doc_index > self.batch_end:
                self.load_new_batch_in_memory()
            for (doc_id, tokens) in zip(self.curr_docids, self.curr_lines):
                if self.batch_size is not None:
                    curr_batch_iter = 0
                    # divide the document to batches according to the batch size
                    while curr_batch_iter < len(tokens):
                        yield LabeledSentence(words=tokens[curr_batch_iter: curr_batch_iter + self.batch_size], tags=[doc_id])
                        curr_batch_iter += self.batch_size
                else:
                    yield doc_id, tokens
                self.curr_index += 1
                # increment only for full docs
                if is_real_doc(doc_id):
                    self.curr_doc_index += 1
                    if self.curr_doc_index % 1000 == 0:
                        info("New Doc: {:5}, Batch End: {:5}".format(self.curr_doc_index, self.batch_end))

def is_real_doc(doc_id):
    return doc_id.find("_") == -1






class ExtendedPVDocumentBatchGenerator(Process):
    def __init__(self, filename_prefix, queue, batch_size=10000, start_file=0, offset=10000):
        """
        batch_size cant be > 10,000 due to a limitation in doc2vec training, 
        None means no batching (only use for inference)
        """
        assert batch_size <= 10000 or batch_size is None
        super(ExtendedPVDocumentBatchGenerator, self).__init__()
        self.queue = queue
        self.offset = offset
        self.filename_prefix = filename_prefix
        self.batch_size = batch_size
        self.cur_file = None
        self.files_loaded = start_file - offset

    def line_processing(self, line, batch_size):
        line_array = tuple(line.split(" "))
        doc_id = line_array[0]
        line_array = line_array[1:]
        if self.is_doc_part_chunk(doc_id):
            return 
        len_line_array = len(line_array)
        curr_batch_iter = 0
        # divide the document to batches according to the batch size
        while curr_batch_iter < len_line_array:
            self.queue.put(LabeledSentence(words=line_array[curr_batch_iter: curr_batch_iter + batch_size], tags=[doc_id]), block=True, timeout=None)
            curr_batch_iter += batch_size
        return

    def run(self):
        while True:
            try:
                if self.cur_file is None:
                    info("Loading new file for index: {}".format(str(self.files_loaded + self.offset)))
                    self.cur_file = open(self.filename_prefix + str(self.files_loaded + self.offset))
                    self.files_loaded += self.offset
                for line in self.cur_file:
                    self.line_processing(line, self.batch_size)
                self.cur_file.close()
                self.cur_file = None
            except IOError:
                self.queue.put(False, block=True, timeout=None)
                info("All files are loaded - last file: {}".format(str(self.files_loaded + self.offset)))
                return
    
    def get_entity_parts(self, entity_id):
        return entity_id.split("_")
    def is_doc(self, entity_id):
        parts = self.get_entity_parts(entity_id)
        if len(parts) == 1:
            return True
        return False
    def is_doc_part(self, entity_id):
        parts = self.get_entity_parts(entity_id)
        if len(parts) == 2:
            return True
        return False
    def is_doc_part_chunk(self, entity_id):
        parts = self.get_entity_parts(entity_id)
        if len(parts) == 3:
            return True
        return False


class BatchWrapper(object):
    def __init__(self, training_preprocessed_files_prefix, buffer_size=10000, batch_size=10000):
        self.q = Queue(maxsize=buffer_size)
        self.p = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, queue=self.q,
                                                  batch_size=batch_size, start_file=0, offset=10000)
        self.p.start()
        self.cur_data = []

    def __iter__(self):
        while True:
            item = self.q.get(block=True)
            if item is False:
                raise StopIteration()
            else:
                yield item

# Doc2vec and SVM Parameters

In [16]:
DOC2VEC_SIZE = 200
DOC2VEC_WINDOW = 2
DOC2VEC_MAX_VOCAB_SIZE = None
DOC2VEC_SAMPLE = 1e-3
DOC2VEC_TYPE = 1
DOC2VEC_HIERARCHICAL_SAMPLE = 0
DOC2VEC_NEGATIVE_SAMPLE_SIZE = 10
DOC2VEC_CONCAT = 0
DOC2VEC_MEAN = 1
DOC2VEC_TRAIN_WORDS = 0
DOC2VEC_EPOCHS = 1 # we do our training manually one epoch at a time
DOC2VEC_MAX_EPOCHS = 8
REPORT_DELAY = 20 # report the progress every x seconds
REPORT_VOCAB_PROGRESS = 100000 # report vocab progress every x documents

In [17]:
BatchClass = BatchWrapper
# BatchClass = ExtendedPVDocumentBatchGeneratorMine

## Create the Doc2vec model

In [18]:
placeholder_model_name = 'doc2vec_size_{}_w_{}_type_{}_concat_{}_mean_{}_trainwords_{}_hs_{}_neg_{}_vocabsize_{}'.format(DOC2VEC_SIZE, 
                                                                DOC2VEC_WINDOW, 
                                                                'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
                                                                DOC2VEC_CONCAT, DOC2VEC_MEAN,
                                                                DOC2VEC_TRAIN_WORDS,
                                                                DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                                                                str(DOC2VEC_MAX_VOCAB_SIZE))
GLOBAL_VARS.DOC2VEC_MODEL_NAME = placeholder_model_name
placeholder_model_name = os.path.join(placeholder_model_name, "epoch_{}")
placeholder_model_name

'doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_{}'

In [19]:
doc2vec_model = Doc2Vec(size=DOC2VEC_SIZE , window=DOC2VEC_WINDOW, min_count=MIN_WORD_COUNT, 
                max_vocab_size= DOC2VEC_MAX_VOCAB_SIZE,
                sample=DOC2VEC_SAMPLE, seed=DOC2VEC_SEED, workers=NUM_CORES,
                # doc2vec algorithm dm=1 => PV-DM, dm=2 => PV-DBOW, PV-DM dictates CBOW for words
                dm=DOC2VEC_TYPE,
                # hs=0 => negative sampling, hs=1 => hierarchical softmax
                hs=DOC2VEC_HIERARCHICAL_SAMPLE, negative=DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                dm_concat=DOC2VEC_CONCAT,
                # would train words with skip-gram on top of cbow, we don't need that for now
                dbow_words=DOC2VEC_TRAIN_WORDS,
                iter=DOC2VEC_EPOCHS)

GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model

## First: construct (or load) the vocabulary
Only needed to be run if you dont already haave at least one epoch computed, otherwise, just set the start_from (below) to the epoch you want to restart from

In [ ]:
%%time
# training_docs_iterator = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, batch_size=10000)
training_docs_iterator = BatchClass(training_preprocessed_files_prefix, batch_size=10000)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX)):
    doc2vec_model.build_vocab(sentences=training_docs_iterator, progress_per=REPORT_VOCAB_PROGRESS)
    doc2vec_model.save(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
else:
    doc2vec_model_vocab_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
    doc2vec_model.reset_from(doc2vec_model_vocab_model)

2017-04-06 17:09:28,018 : INFO : collecting all words and their counts
2017-04-06 17:09:28,019 : INFO : Loading new file for index: 0
2017-04-06 17:09:28,029 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-04-06 17:10:12,630 : INFO : Loading new file for index: 10000
2017-04-06 17:10:54,587 : INFO : Loading new file for index: 20000
2017-04-06 17:11:06,135 : INFO : PROGRESS: at example #100000, processed 401162100 words (4089108/s), 1195495 word types, 83019 tags
2017-04-06 17:11:36,162 : INFO : Loading new file for index: 30000
2017-04-06 17:12:17,810 : INFO : Loading new file for index: 40000
2017-04-06 17:12:31,298 : INFO : PROGRESS: at example #200000, processed 809962086 words (4800319/s), 2059298 word types, 165183 tags
2017-04-06 17:12:57,970 : INFO : Loading new file for index: 50000
2017-04-06 17:13:39,090 : INFO : Loading new file for index: 60000
2017-04-06 17:13:54,647 : INFO : PROGRESS: at example #300000, processed 1219476016 words (49

2017-04-06 17:55:26,305 : INFO : Loading new file for index: 670000
2017-04-06 17:56:07,657 : INFO : Loading new file for index: 680000
2017-04-06 17:56:24,758 : INFO : PROGRESS: at example #3300000, processed 13381584277 words (4785822/s), 16297546 word types, 2728397 tags
2017-04-06 17:56:48,959 : INFO : Loading new file for index: 690000
2017-04-06 17:57:29,754 : INFO : Loading new file for index: 700000
2017-04-06 17:57:50,366 : INFO : PROGRESS: at example #3400000, processed 13789892400 words (4769589/s), 16625117 word types, 2810619 tags
2017-04-06 17:58:11,770 : INFO : Loading new file for index: 710000
2017-04-06 17:58:52,378 : INFO : Loading new file for index: 720000
2017-04-06 17:59:14,123 : INFO : PROGRESS: at example #3500000, processed 14198166356 words (4874624/s), 16974797 word types, 2892753 tags
2017-04-06 17:59:33,657 : INFO : Loading new file for index: 730000
2017-04-06 18:00:15,204 : INFO : Loading new file for index: 740000
2017-04-06 18:00:39,247 : INFO : PROGRE

2017-04-06 18:36:41,245 : INFO : Loading new file for index: 1280000
2017-04-06 18:36:55,399 : INFO : PROGRESS: at example #6200000, processed 25133398178 words (4911402/s), 25099211 word types, 5126009 tags
2017-04-06 18:37:06,676 : INFO : Loading new file for index: 1290000
2017-04-06 18:37:06,677 : INFO : All files are loaded - last file: 1290000
2017-04-06 18:37:14,859 : INFO : collected 25169847 word types and 5145300 unique tags from a corpus of 6223486 examples and 25229940994 words
2017-04-06 18:37:33,950 : INFO : min_count=200 retains 446839 unique words (drops 24723008)
2017-04-06 18:37:33,951 : INFO : min_count leaves 25008102081 word corpus (99% of original 25229940994)
2017-04-06 18:37:35,109 : INFO : deleting the raw counts dictionary of 25169847 items
2017-04-06 18:37:38,137 : INFO : sample=0.001 downsamples 31 most-common words
2017-04-06 18:37:38,138 : INFO : downsampling leaves estimated 17192719202 word corpus (68.7% of prior 25008102081)
2017-04-06 18:37:38,138 : IN

CPU times: user 1h 26min 17s, sys: 2min 7s, total: 1h 28min 25s
Wall time: 1h 30min 16s


In [21]:
# vocab_counts = {k:doc2vec_model.vocab[k].count for k in doc2vec_model.vocab.keys()}
# dd = sorted(vocab_counts, key=vocab_counts.get)

## Actual Training, validation and Metrics Loop

In [21]:
doc2vec_model.min_alpha = 0.025
DOC2VEC_ALPHA_DECREASE = 0.001

In [23]:
doc2vec_model.workers = NUM_CORES

In [ ]:
%%time
# when resuming, resume from an epoch with a previously created doc2vec model to get the learning rate right
start_from = 8
for epoch in range(start_from, DOC2VEC_MAX_EPOCHS+1):
    GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
    info("****************** Epoch {} --- Working on {} *******************".format(epoch, GLOBAL_VARS.MODEL_NAME))
    
    # if we have the model, just load it, otherwise train the previous model
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
        doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        doc2vec_model.workers = NUM_CORES
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
    else:
        # train the doc2vec model
#         training_docs_iterator = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, batch_size=10000)
        training_docs_iterator = BatchClass(training_preprocessed_files_prefix, batch_size=10000)
        %time doc2vec_model.train(sentences=training_docs_iterator, report_delay=REPORT_DELAY)
        doc2vec_model.alpha -= DOC2VEC_ALPHA_DECREASE  # decrease the learning rate
        doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay
        ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME))
        doc2vec_model.save(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
        
        # get the word2vec analogy accuracy score
#         word2vec_result = doc2vec_model.accuracy(word2vec_questions_file, restrict_vocab=None)
#         epoch_word2vec_metrics.append(word2vec_result)
#         pickle.dump(word2vec_result, open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME,
#                                                        WORD2VEC_METRICS_FILENAME), 'w'))

    # only do the inference for higher epochs, as inference usually takes as much time as the actual training
    if epoch > 7:
        # Validation Embeddings
        info('Getting Validation Embeddings')
        Xv = get_extended_docs_only_with_inference(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix)

2017-04-06 19:07:09,827 : INFO : ****************** Epoch 1 --- Working on doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_1 *******************
2017-04-06 19:07:09,991 : INFO : training model with 8 workers on 446839 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=10
2017-04-06 19:07:09,992 : INFO : Loading new file for index: 0
2017-04-06 19:07:09,998 : INFO : expecting 6223486 sentences, matching count from corpus used for vocabulary survey
2017-04-06 19:07:11,008 : INFO : PROGRESS: at 0.00% examples, 527528 words/s, in_qsize 15, out_qsize 1
2017-04-06 19:07:31,010 : INFO : PROGRESS: at 0.10% examples, 778608 words/s, in_qsize 8, out_qsize 0
2017-04-06 19:07:51,012 : INFO : PROGRESS: at 0.19% examples, 799442 words/s, in_qsize 15, out_qsize 0
2017-04-06 19:08:11,014 : INFO : PROGRESS: at 0.29% examples, 812431 words/s, in_qsize 14, out_qsize 0
2017-04-06 19:08:31,021 : INFO : PROGRESS: at 0.39% examples, 817734 words/s, i

2017-04-06 19:30:16,849 : INFO : Loading new file for index: 90000
2017-04-06 19:30:31,392 : INFO : PROGRESS: at 6.93% examples, 853828 words/s, in_qsize 16, out_qsize 0
2017-04-06 19:30:51,400 : INFO : PROGRESS: at 7.03% examples, 854115 words/s, in_qsize 15, out_qsize 0
2017-04-06 19:31:11,400 : INFO : PROGRESS: at 7.14% examples, 854447 words/s, in_qsize 13, out_qsize 0
2017-04-06 19:31:31,408 : INFO : PROGRESS: at 7.24% examples, 854665 words/s, in_qsize 15, out_qsize 0
2017-04-06 19:31:51,410 : INFO : PROGRESS: at 7.34% examples, 854989 words/s, in_qsize 13, out_qsize 0
2017-04-06 19:32:11,417 : INFO : PROGRESS: at 7.44% examples, 855171 words/s, in_qsize 15, out_qsize 0
2017-04-06 19:32:31,419 : INFO : PROGRESS: at 7.54% examples, 855266 words/s, in_qsize 13, out_qsize 1
2017-04-06 19:32:49,541 : INFO : Loading new file for index: 100000
2017-04-06 19:32:51,423 : INFO : PROGRESS: at 7.64% examples, 855558 words/s, in_qsize 14, out_qsize 0
2017-04-06 19:33:11,428 : INFO : PROGRESS

2017-04-06 19:54:51,708 : INFO : PROGRESS: at 14.32% examples, 863323 words/s, in_qsize 15, out_qsize 1
2017-04-06 19:55:11,720 : INFO : PROGRESS: at 14.42% examples, 863416 words/s, in_qsize 15, out_qsize 0
2017-04-06 19:55:31,720 : INFO : PROGRESS: at 14.53% examples, 863567 words/s, in_qsize 12, out_qsize 0
2017-04-06 19:55:51,720 : INFO : PROGRESS: at 14.63% examples, 863684 words/s, in_qsize 14, out_qsize 1
2017-04-06 19:55:52,958 : INFO : Loading new file for index: 190000
2017-04-06 19:56:11,732 : INFO : PROGRESS: at 14.73% examples, 863749 words/s, in_qsize 14, out_qsize 2
2017-04-06 19:56:31,739 : INFO : PROGRESS: at 14.84% examples, 863868 words/s, in_qsize 16, out_qsize 0
2017-04-06 19:56:51,741 : INFO : PROGRESS: at 14.94% examples, 863903 words/s, in_qsize 15, out_qsize 1
2017-04-06 19:57:11,747 : INFO : PROGRESS: at 15.04% examples, 863961 words/s, in_qsize 15, out_qsize 0
2017-04-06 19:57:31,748 : INFO : PROGRESS: at 15.15% examples, 863999 words/s, in_qsize 14, out_qsiz

2017-04-06 20:19:12,101 : INFO : PROGRESS: at 21.74% examples, 867671 words/s, in_qsize 16, out_qsize 0
2017-04-06 20:19:32,105 : INFO : PROGRESS: at 21.84% examples, 867723 words/s, in_qsize 15, out_qsize 0
2017-04-06 20:19:52,105 : INFO : PROGRESS: at 21.94% examples, 867751 words/s, in_qsize 14, out_qsize 0
2017-04-06 20:20:12,112 : INFO : PROGRESS: at 22.05% examples, 867767 words/s, in_qsize 15, out_qsize 0
2017-04-06 20:20:32,107 : INFO : PROGRESS: at 22.15% examples, 867801 words/s, in_qsize 15, out_qsize 0
2017-04-06 20:20:52,114 : INFO : PROGRESS: at 22.25% examples, 867802 words/s, in_qsize 15, out_qsize 0
2017-04-06 20:21:12,117 : INFO : PROGRESS: at 22.35% examples, 867816 words/s, in_qsize 16, out_qsize 2
2017-04-06 20:21:24,512 : INFO : Loading new file for index: 290000
2017-04-06 20:21:32,115 : INFO : PROGRESS: at 22.46% examples, 867910 words/s, in_qsize 14, out_qsize 0
2017-04-06 20:21:52,119 : INFO : PROGRESS: at 22.56% examples, 867967 words/s, in_qsize 15, out_qsiz

2017-04-06 20:43:32,426 : INFO : PROGRESS: at 29.18% examples, 870098 words/s, in_qsize 16, out_qsize 0
2017-04-06 20:43:52,427 : INFO : PROGRESS: at 29.28% examples, 870154 words/s, in_qsize 16, out_qsize 0
2017-04-06 20:44:12,429 : INFO : PROGRESS: at 29.39% examples, 870209 words/s, in_qsize 15, out_qsize 1
2017-04-06 20:44:17,134 : INFO : Loading new file for index: 380000
2017-04-06 20:44:32,436 : INFO : PROGRESS: at 29.50% examples, 870234 words/s, in_qsize 16, out_qsize 0
2017-04-06 20:44:52,439 : INFO : PROGRESS: at 29.60% examples, 870269 words/s, in_qsize 15, out_qsize 1
2017-04-06 20:45:12,458 : INFO : PROGRESS: at 29.70% examples, 870349 words/s, in_qsize 16, out_qsize 0
2017-04-06 20:45:32,470 : INFO : PROGRESS: at 29.80% examples, 870383 words/s, in_qsize 13, out_qsize 1
2017-04-06 20:45:52,476 : INFO : PROGRESS: at 29.91% examples, 870392 words/s, in_qsize 15, out_qsize 0
2017-04-06 20:46:12,481 : INFO : PROGRESS: at 30.00% examples, 870431 words/s, in_qsize 16, out_qsiz

...

2017-04-08 13:20:25,041 : INFO : PROGRESS: at 89.99% examples, 893821 words/s, in_qsize 12, out_qsize 0
2017-04-08 13:20:37,849 : INFO : Loading new file for index: 1160000
2017-04-08 13:20:45,043 : INFO : PROGRESS: at 90.09% examples, 893828 words/s, in_qsize 15, out_qsize 1
2017-04-08 13:21:05,045 : INFO : PROGRESS: at 90.19% examples, 893837 words/s, in_qsize 15, out_qsize 0
2017-04-08 13:21:25,047 : INFO : PROGRESS: at 90.30% examples, 893844 words/s, in_qsize 15, out_qsize 1
2017-04-08 13:21:45,053 : INFO : PROGRESS: at 90.40% examples, 893859 words/s, in_qsize 15, out_qsize 0
2017-04-08 13:22:05,074 : INFO : PROGRESS: at 90.51% examples, 893871 words/s, in_qsize 14, out_qsize 3
2017-04-08 13:22:25,069 : INFO : PROGRESS: at 90.61% examples, 893883 words/s, in_qsize 15, out_qsize 0
2017-04-08 13:22:45,071 : INFO : PROGRESS: at 90.72% examples, 893892 words/s, in_qsize 15, out_qsize 0
2017-04-08 13:23:05,076 : INFO : PROGRESS: at 90.82% examples, 893905 words/s, in_qsize 15, out_qsi

2017-04-08 13:44:45,438 : INFO : PROGRESS: at 97.67% examples, 894297 words/s, in_qsize 16, out_qsize 0
2017-04-08 13:45:05,446 : INFO : PROGRESS: at 97.77% examples, 894305 words/s, in_qsize 15, out_qsize 0
2017-04-08 13:45:11,013 : INFO : Loading new file for index: 1260000
2017-04-08 13:45:25,462 : INFO : PROGRESS: at 97.88% examples, 894308 words/s, in_qsize 14, out_qsize 2
2017-04-08 13:45:45,472 : INFO : PROGRESS: at 97.98% examples, 894308 words/s, in_qsize 11, out_qsize 1
2017-04-08 13:46:05,467 : INFO : PROGRESS: at 98.08% examples, 894332 words/s, in_qsize 15, out_qsize 0
2017-04-08 13:46:25,468 : INFO : PROGRESS: at 98.19% examples, 894337 words/s, in_qsize 16, out_qsize 0
2017-04-08 13:46:45,469 : INFO : PROGRESS: at 98.30% examples, 894339 words/s, in_qsize 15, out_qsize 1
2017-04-08 13:47:05,480 : INFO : PROGRESS: at 98.41% examples, 894351 words/s, in_qsize 16, out_qsize 1
2017-04-08 13:47:25,483 : INFO : PROGRESS: at 98.51% examples, 894347 words/s, in_qsize 14, out_qsi

CPU times: user 1d 10h 50min 58s, sys: 20min 6s, total: 1d 11h 11min 4s
Wall time: 5h 20min 28s


2017-04-08 13:52:11,480 : INFO : storing numpy array 'syn1neg' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks_levels_0_1/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.syn1neg.npy
2017-04-08 13:52:11,699 : INFO : not storing attribute syn0norm
2017-04-08 13:52:11,700 : INFO : storing numpy array 'syn0' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_extended_abs_desc_claims_full_chunks_levels_0_1/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.syn0.npy
2017-04-08 13:52:11,964 : INFO : not storing attribute cum_table
2017-04-08 13:53:05,082 : INFO : Getting Validation Embeddings


NameError: name 'classifications' is not defined